In [ ]:
from kaggle import api
api.competition_download_file('titanic', "train.csv", path="./dataset")
api.competition_download_file('titanic', "test.csv", path="./dataset")

In [ ]:
from torch.utils.data import Dataset, DataLoader, random_split
from torch import Tensor
import pandas as pd

class TitanicDataset(Dataset):
    def __init__(self, file: str):
        self.data = pd.read_csv(file)
        # 补充空值
        self.data.loc[:, 'Age'] = pd.to_numeric(self.data.loc[:, 'Age'], errors='coerce')
        mean_age = self.data.loc[:, 'Age'].mean()
        self.data[self.data.loc[:, 'Age'].isna()] = int(mean_age)

        # 处理性别
        self.data.loc[:, 'Sex'] = self.data.loc[:, 'Sex'].map({'male': 0, 'female': 1})
        self.data = self.data[self.data.loc[:, "Embarked"].notnull()]
        self.data.loc[:, "Embarked"] = self.data.loc[:, "Embarked"].map({"S": 0, "C": 1, "Q": 2})
        self.data.drop([ 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
    
    def __len__(self):
        return len(self.data)
        
    def __getitem__(self, index):
        item = self.data.iloc[index]
        id = item.loc['PassengerId']
        if 'Survived' not in item:
            item = item.drop(["PassengerId"])
            return id, Tensor(item), Tensor([0, 0])
        label = item.loc['Survived']
        label = [1 if label == 0 else 0, 1 if label == 1 else 0]
        item = item.drop(["PassengerId",'Survived'])
        return id, Tensor(item), Tensor(label)

pd.get_dummies

data = TitanicDataset("./dataset/train.csv")
print(len(data), data[0])
train_set, validate_set = random_split(data, [0.9, 0.1])
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
validate_loader = DataLoader(validate_set, batch_size=32, shuffle=True)
test_set_loader = DataLoader(TitanicDataset("./dataset/test.csv"), batch_size=32, shuffle=False)

In [ ]:
import torch
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Running on the GPU")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Running on the MPS")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

In [ ]:
from torch import nn

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.output = nn.Sequential(
            nn.Linear(7, 50),
            nn.ReLU(),
            nn.Linear(50,50),
            nn.ReLU(),
            nn.Linear(50, 2),
            nn.Softmax(dim=1)
        )
    
    def forward(self, input):
        return self.output(input)

net = Net()
net.to(device=device)

In [ ]:
epoch = 100
lr = 0.01
batch_size = 100

optimizer = torch.optim.SGD(net.parameters(), lr=lr)
loss_func = torch.nn.CrossEntropyLoss()

In [ ]:
from tqdm import tqdm

for i in range(epoch):
    for _, p, labels in tqdm(train_loader):
        p = p.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        output = net(p)
        loss = loss_func(output, labels)
        print(loss)
        loss.backward()
        optimizer.step()

In [129]:
correct = 0
total = 0

with torch.no_grad():
  net.eval()
  for _, p, labels in validate_loader:
    p, labels = p.to(device), labels.to(device)
    output = net(p)
    # calculate accuracy
    print(output)
    print(torch.argmax(output, dim=0), torch.argmax(labels, dim=0))
    correct += (torch.argmax(output, dim=1) == torch.argmax(labels, dim=1)).sum().item()
    total += labels.size(0)
print("Accuracy: ", round(correct/total, 3))

tensor([[0.6741, 0.3259],
        [0.6741, 0.3259],
        [0.6741, 0.3259],
        [0.6741, 0.3259],
        [0.6741, 0.3259],
        [0.6741, 0.3259],
        [0.6741, 0.3259],
        [0.6741, 0.3259],
        [0.6741, 0.3259],
        [0.6741, 0.3259],
        [0.6741, 0.3259],
        [0.6741, 0.3259],
        [0.6741, 0.3259],
        [0.6741, 0.3259],
        [0.6741, 0.3259],
        [0.6741, 0.3259],
        [0.6741, 0.3259],
        [0.6741, 0.3259],
        [0.6741, 0.3259],
        [0.6741, 0.3259],
        [0.6741, 0.3259],
        [0.6741, 0.3259],
        [0.6741, 0.3259],
        [0.6741, 0.3259],
        [0.6741, 0.3259],
        [0.6741, 0.3259],
        [0.6741, 0.3259],
        [0.6741, 0.3259],
        [0.6741, 0.3259],
        [0.6741, 0.3259],
        [0.6741, 0.3259],
        [0.6741, 0.3259]], device='mps:0')
tensor([0, 0], device='mps:0') tensor([0, 1], device='mps:0')
tensor([[0.6741, 0.3259],
        [0.6741, 0.3259],
        [0.6741, 0.3259],
        [0.

/var/folders/kx/pq42k6ld3kx9sttlst91g2l00000gp/T/ipykernel_99098/1501739964.py:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return id, Tensor(item), Tensor(label)
/var/folders/kx/pq42k6ld3kx9sttlst91g2l00000gp/T/ipykernel_99098/1501739964.py:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return id, Tensor(item), Tensor(label)
/var/folders/kx/pq42k6ld3kx9sttlst91g2l00000gp/T/ipykernel_99098/1501739964.py:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by pos

In [130]:
with torch.no_grad():
    net.eval()
    with open("./dataset/submission.csv", "w") as f:
        f.write("PassengerId,Survived\n")
        for id, p, _ in test_set_loader:
            p = p.to(device)
            output = net(p)
            survival = torch.argmax(output, dim=1)
            for i in range(len(survival)):
                f.write(f"{id[i].int()},{survival[i].int()}\n")
                print(id[i].int(), survival[i].int())

/var/folders/kx/pq42k6ld3kx9sttlst91g2l00000gp/T/ipykernel_99098/1501739964.py:27: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return id, Tensor(item), Tensor([0, 0])
/var/folders/kx/pq42k6ld3kx9sttlst91g2l00000gp/T/ipykernel_99098/1501739964.py:27: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return id, Tensor(item), Tensor([0, 0])
/var/folders/kx/pq42k6ld3kx9sttlst91g2l00000gp/T/ipykernel_99098/1501739964.py:27: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by p

tensor(892, dtype=torch.int32) tensor(0, device='mps:0', dtype=torch.int32)
tensor(893, dtype=torch.int32) tensor(0, device='mps:0', dtype=torch.int32)
tensor(894, dtype=torch.int32) tensor(0, device='mps:0', dtype=torch.int32)
tensor(895, dtype=torch.int32) tensor(0, device='mps:0', dtype=torch.int32)
tensor(896, dtype=torch.int32) tensor(0, device='mps:0', dtype=torch.int32)
tensor(897, dtype=torch.int32) tensor(0, device='mps:0', dtype=torch.int32)
tensor(898, dtype=torch.int32) tensor(0, device='mps:0', dtype=torch.int32)
tensor(899, dtype=torch.int32) tensor(0, device='mps:0', dtype=torch.int32)
tensor(900, dtype=torch.int32) tensor(0, device='mps:0', dtype=torch.int32)
tensor(901, dtype=torch.int32) tensor(0, device='mps:0', dtype=torch.int32)
tensor(30, dtype=torch.int32) tensor(0, device='mps:0', dtype=torch.int32)
tensor(903, dtype=torch.int32) tensor(0, device='mps:0', dtype=torch.int32)
tensor(904, dtype=torch.int32) tensor(0, device='mps:0', dtype=torch.int32)
tensor(905, d

/var/folders/kx/pq42k6ld3kx9sttlst91g2l00000gp/T/ipykernel_99098/1501739964.py:27: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return id, Tensor(item), Tensor([0, 0])
/var/folders/kx/pq42k6ld3kx9sttlst91g2l00000gp/T/ipykernel_99098/1501739964.py:27: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return id, Tensor(item), Tensor([0, 0])
/var/folders/kx/pq42k6ld3kx9sttlst91g2l00000gp/T/ipykernel_99098/1501739964.py:27: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by p

tensor(0, device='mps:0', dtype=torch.int32)
tensor(1057, dtype=torch.int32) tensor(0, device='mps:0', dtype=torch.int32)
tensor(1058, dtype=torch.int32) tensor(0, device='mps:0', dtype=torch.int32)
tensor(1059, dtype=torch.int32) tensor(0, device='mps:0', dtype=torch.int32)
tensor(30, dtype=torch.int32) tensor(0, device='mps:0', dtype=torch.int32)
tensor(1061, dtype=torch.int32) tensor(0, device='mps:0', dtype=torch.int32)
tensor(30, dtype=torch.int32) tensor(0, device='mps:0', dtype=torch.int32)
tensor(1063, dtype=torch.int32) tensor(0, device='mps:0', dtype=torch.int32)
tensor(1064, dtype=torch.int32) tensor(0, device='mps:0', dtype=torch.int32)
tensor(30, dtype=torch.int32) tensor(0, device='mps:0', dtype=torch.int32)
tensor(1066, dtype=torch.int32) tensor(0, device='mps:0', dtype=torch.int32)
tensor(1067, dtype=torch.int32) tensor(0, device='mps:0', dtype=torch.int32)
tensor(1068, dtype=torch.int32) tensor(0, device='mps:0', dtype=torch.int32)
tensor(1069, dtype=torch.int32) tenso

/var/folders/kx/pq42k6ld3kx9sttlst91g2l00000gp/T/ipykernel_99098/1501739964.py:27: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return id, Tensor(item), Tensor([0, 0])
/var/folders/kx/pq42k6ld3kx9sttlst91g2l00000gp/T/ipykernel_99098/1501739964.py:27: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return id, Tensor(item), Tensor([0, 0])
/var/folders/kx/pq42k6ld3kx9sttlst91g2l00000gp/T/ipykernel_99098/1501739964.py:27: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by p